In [1]:
# 클로저가 변수 스코프와 상호작용 하는 법을 알자

In [5]:
# 클로저란 자기 자신이 정의된 스코프에 존재하는 변수를 참조하는 함수다.
# 여기 예를 들어보자
def my_name(name):
    _name = name
    def print():
        return _name
    return print()
    
print(my_name('minsoo'))

minsoo


In [6]:
# 일반적으로 print함수 내부에 _name 변수가 정의 되어있지 않으므로 print함수가 _name을 return 할 수 없다고 생각할 수 있다.
# 하지만 클로저 함수의 특성으로 인해 이를 가능하게 해준다.
# 좀 더 어려운 구현 방법으로는 아래와 같은 방법이 있다.
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

numbers = [8,3,1,2,5,4,7,6]
group = {2, 3, 7, 5}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [7]:
# 위의 함수가 제대로 작동할 수 있는 이유는 크게 3가지다
# 1. 클로저 함수의 특성으로 helper 함수가 group 인수에 접근할 수 있다.
# 2. 파이썬에서 함수는 일급 객체(즉, 함수를 직접 참조하거나 변수에 넣을 수 있고, 다른 함수의 매개변수로 넣을 수도 있다. 그래서 클로저 함수를 key 인수로 넣을 수 있다.)
# 3. 파이썬 튜플에서는 인덱스 0을 비교하여 우선순위를 나누고, 후에 인덱스 1을 비교해서 나눈다. 그런 순서로 2,3,4 번째 인덱스로 나누는 것을 반복한다.

In [8]:
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print('Found : ', found)
print(numbers)

Found :  False
[2, 3, 5, 7, 1, 4, 6, 8]


In [9]:
# found 값이 True로 나와야할 것 같은데 이상하게 False가 나왔다. 그 이유를 생각해볼 필요가 있다.
# 표현식에서 변수를 참조할 때, 파이썬 인터프리터는 참조를 해결하기 위해, 다음과 같은 순서로 스코프(유효범위)를 탐색한다.

# 1. 현재 함수의 스코프
# 2. (현재 스코프를 담고 있는 다른 함수 같은)감싸고 있는 스코프
# 3. 코드를 포함하고 있는 모듈의 스코프(전역 스코프라고도 한다.)
# 4. (len이나 str 같은 함수를 담고 있는 스코프) 내장 스코프

# 이 와중에 어느 스코프에도 참조한 이름으로 된 변수가 없다면 NameError를 일으킨다.

In [10]:
# 데이터 얻어오기
# 파이썬3에는 클로저에서 데이터를 얻어오는 특별한 문법이 있다.
# nonlocal문은 특정 변수 이름에 할당할 때 스코프 탐색이 일어나야함을 나타낸다.
# 유일한 제약은 nonlocal이 (전역 변수의 오염을 피하기 위해)모듈 수준 스코프까지 탐색할 수 없다는 점이다.
def sort_priority3(numbes, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0,x)
        return (1,x)
    numbers.sort(key=helper)
    return found

In [11]:
# nonlocal 문은 클로저에서 데이터를 다른 스코프에 할당하는 시점을 알아보기 쉽게 해준다.
# 변수 할당이 모듈 스코프에 직접 들어가게 하는 global 문을 보완한다.
found = sort_priority3(numbers, group)
print('Found : ', found)
print(numbers)

Found :  True
[2, 3, 5, 7, 1, 4, 6, 8]


In [16]:
# 하지만 nonlocal문은 안티패턴이므로 간단한 함수 이외에는 사용하면 안된다.
# nonlocal을 사용했을 때 나타나는 부작용은 알아내기 굉장히 힘들다.
# nonlocal을 사용할 때 갑자기 복잡해지기 시작하면 헬퍼 클래스로 상태를 감싸는 것이 좋다.
# 조금 더 길어지지만 이해하기는 훨씬 쉽다.
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True